In [114]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import os
import os.path
import statistics as stat
import math

In [115]:
def forward (V, a, b, initial_distribution):
    ## probability of being in state j at trial t, output is T*M matrix, T is number of trials, M is number of states
    alpha = np.zeros((V.shape[0],a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
    return alpha

In [116]:
def backward (V, a, b):
    ## time-reversed version of forward algoritms
    ## output is T*M matrix, T is number of trials, M is number of states
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
    return beta

In [117]:
def baum_welch(V, a, b, initial_distribution, n_iter=100, parameterTying = True):
    M = a.shape[0] ## number of states
    T = len(V) ##  number of trials
 
    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
        
        ## xi = p (s(t) = i, s(t+1) = j | observation, model)
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        ## gamma = p (s(t)=i |observation, model) = sum_j^M {xi}
        gamma = np.sum(xi, axis=1)
        
        transMatrixNum = np.sum(xi, 2)
        transMatrixDem = np.sum(gamma, axis=1).reshape((-1, 1))
        
        ## a is the transition matrix being iteratively optmized and will eventually converge
        a = transMatrixNum/transMatrixDem

    ## tie parameter here
    if parameterTying:
        a[0,1] = (a[0,1] + a[0,2])/2
        a[0,2] = a[0,1]

        a[1,1] = (a[1,1]+a[2,2])/2
        a[2,2] = a[1,1]

        a[1,0] = (a[1,0]+a[2,0])/2
        a[2,0] = a[1,0]
    
    return a

In [118]:
def viterbi(V, a, b, initial_distribution):
    T = V.shape[0] ## number of trial
    M = a.shape[0] ## number of states

    omega = np.zeros((T, M))
    #omega[0, :] = np.log(initial_distribution * b[:, V[0]]) # prior
    omega[0, :] = (initial_distribution * b[:, V[0]]) # prior

    prev = np.zeros((T - 1, M)) 
 
    for t in range(1, T):
        for j in range(M):
            # Same as Forward Probability
            probability = omega[t - 1] + np.log(a[:, j]) + np.log(b[j, V[t]])
            
            # This is our most probable state given previous state at time t (1)
            prev[t - 1, j] = np.argmax(probability)
 
            # This is the probability of the most probable state (2)
            omega[t, j] = np.max(probability)
    

    ## shape of omega is 300x3
 
    # Path Array
    S = np.zeros(T)
 
    # Find the most probable last hidden state
    last_state = np.argmax(omega[T - 1, :])
 
    S[0] = last_state
    backtrack_index = 1
    for i in range(T - 2, -1, -1):
        S[backtrack_index] = prev[i, int(last_state)]
        last_state = prev[i, int(last_state)]
        backtrack_index += 1
 
    # Flip the path array since we were backtracking
    result = np.flip(S, axis=0)
    
 
    return result

In [119]:
def fitHMM (choice):  ### the input is an array of choice, make sure choices are coded as 0 and 1
    
    ## emission probs (p(choice|state))
    b = np.array([[1/2,1/2],[1,0],[0,1]])

    # initial prob distribution
    # start in exploration state
    initial_distribution = np.array((1,0,0))

    ## reseeding to find the real optimized transition matrix
    reseed = 10
    count = -1
    optimizedA = []

    while True:
        count += 1

        ### initialize a random transition matrix A
        rand1 = np.random.random()
        rand2 = np.random.random()
        a = np.array([[(1-rand1),rand1/2,rand1/2],[1-rand2,rand2,0,],[1-rand2,0,rand2]])

        ### using forward-backward algorithms (baum-welch) to find transition matrix
        ### if examining two states (explore vs. exploit), parameterTying =True. THis default
        ### if examining three states (biased exploitation), parameterTying = False
        transMatrix=baum_welch(choice,a,b,initial_distribution, n_iter =100) #matrix A and B
        
        ## reseeding to check if this is the global minimum
        flatMat = np.ndarray.flatten(transMatrix)
        
        if flatMat[0] <= 0.5 or flatMat[3] >= 0.5:
            print ('matrix criteria not reached')
            if count == reseed:
                break
            else:
                continue
        else:
            if count == 0:
                    optimizedA.append (flatMat)

            elif count == reseed:
                print ("break because it has reached reeding limit *********")
                break
            else:
                for m in range (len(optimizedA)):
                    if np.allclose (flatMat, optimizedA[m], rtol=0.01, atol=0.01, equal_nan=False) == False:
                        optimizedA.append (flatMat)  
                    else:
                        break   
                    break
                break

    print (transMatrix)
    explorePotential= transMatrix[0,0]
    exploitPotential = transMatrix[1,1]

    ## fit back to each session using viterbi algorithms
    stateLabel = viterbi(choice,transMatrix,b,initial_distribution).tolist()

    return stateLabel, explorePotential, exploitPotential


In [120]:
df = pd.read_csv('/Users/sijinchen/Desktop/HMM python/test.csv')  ## file path
choice = df['choice'].values-1   ## make sure the choices are 0 and 1

stateLabel, explorePotential, exploitPotential = fitHMM (choice)

print (stateLabel)
print(explorePotential)
print (exploitPotential)

[[0.78200881 0.1089956  0.1089956 ]
 [0.35825665 0.64174335 0.        ]
 [0.35825665 0.         0.64174335]]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 

/var/folders/4c/qtyzwxns1jz7zlm880prvytw0000gn/T/ipykernel_11158/3447853257.py:14: RuntimeWarning: divide by zero encountered in log
  probability = omega[t - 1] + np.log(a[:, j]) + np.log(b[j, V[t]])
